In [72]:
import pandas as pd
import numpy as np
import json
import pickle
import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import csv
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import math

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kratikakothari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kratikakothari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [113]:
model = Word2Vec.load('word2vec_cl_new_ng7.model')

In [114]:
model.init_sims(replace=True)

In [115]:
with open('culinaryDB_new_recipes.pkl', 'rb') as f:
    id_ingredients = pickle.load(f)
with open('culinaryDB_new_vectors.pkl', 'rb') as f:
    id_vector = pickle.load(f)
with open('culinaryDB_new_names.pkl', 'rb') as f:
    id_names = pickle.load(f)

In [120]:
def findSimilarIngredients(current_ingredient,number_of_items=10):
     
    current_vector = model.wv.get_vector(current_ingredient)
#     current_vector = _l2_norm(current_vector,replace=False)
    vectors = []
    vectors = model.wv.vectors
#     vectors=_l2_norm(model.wv.vectors,replace=False)
    distances = np.dot(vectors,current_vector)
    sorted_distances = gensim.matutils.argsort(distances, topn=number_of_items+1, reverse=True)
    smallest_distances = sorted_distances[:number_of_items+1]
    similar_ingredients = []
    for i in smallest_distances:
        if i != model.wv.vocab[current_ingredient].index:
            similar_ingredients.append(model.wv.index2word[i])
    return similar_ingredients

In [121]:
def findComplimentaryIngredients(current_ingredient, number_of_items=10, useNorm=False):
    
    current_vector = model.wv.get_vector(current_ingredient)
    vectors = []
    vectors=model.trainables.syn1neg
    distances = np.dot(vectors,current_vector)
    sorted_distances = gensim.matutils.argsort(distances, topn=number_of_items+1, reverse=True)
    smallest_distances = sorted_distances[:number_of_items+1]
    similar_ingredients = []
    for i in smallest_distances:
        if i != model.wv.vocab[current_ingredient].index:
            similar_ingredients.append(model.wv.index2word[i])
    return similar_ingredients    

In [122]:
def suggest_recipe(current_ingredients,id_vector,id_ingredients,id_names):
    n = len(current_ingredients)
    current_sum = np.zeros(100)
    for ingredient in current_ingredients:
        vector = model.wv[ingredient]
        current_sum = np.add(current_sum,vector)
    current_sum = current_sum/n
    current_sum = current_sum.reshape(1,-1)
    id_cos = []
    for recipe_id in id_vector:
        vector = id_vector[recipe_id]
        flag = 0
        for v in vector:
            if math.isnan(v):
                flag = 1
                break
        if flag == 1:
            continue
        x = cosine_similarity(current_sum,vector.reshape(1,-1))
        id_cos.append((x,recipe_id))
    id_cos = sorted(id_cos,reverse=True)
    count = 0
    for i in id_cos:
        print(id_names[i[1]])
        print(id_ingredients[i[1]])
        count += 1
        if(count>5):
            break